In [4]:
# First, install the required packages
#!pip install sqlalchemy psycopg2-binary

### Step1: Imports and config

In [9]:
import os
import re
import pandas as pd
from sqlalchemy import create_engine, text

# 1) Path to raw CSV
CSV_PATH = "/Users/zoelin/Documents/GitHub/boston-property-valuation-intelligence/data/raw/fy2024_property_assessment.csv"

# 2) Postgres connection (edit username/password if needed)
# If you set a password during install, put it here.
PG_USER = "zoelin"
PG_PASSWORD = ""   # e.g. "your_password"
PG_HOST = "localhost"
PG_PORT = 5432
PG_DB = "boston_property"

engine = create_engine(
    f"postgresql+psycopg2://{PG_USER}:{PG_PASSWORD}@{PG_HOST}:{PG_PORT}/{PG_DB}"
)

print("CSV exists:", os.path.exists(CSV_PATH))

CSV exists: True


### Step2: read only headers + normalize column names

In [10]:
# Read only the header row first (super fast)
df_head = pd.read_csv(CSV_PATH, nrows=5)
raw_cols = list(df_head.columns)

def normalize_col(c: str) -> str:
    c = c.strip().lower()
    c = re.sub(r"\s+", "_", c)          # spaces -> underscore
    c = re.sub(r"[^a-z0-9_]", "", c)    # remove weird chars
    c = re.sub(r"_+", "_", c)           # collapse __
    return c

norm_cols = [normalize_col(c) for c in raw_cols]

# Check duplicates after normalization
dupes = pd.Series(norm_cols).duplicated(keep=False)
if dupes.any():
    print("⚠️ Duplicate normalized columns found:")
    print(pd.DataFrame({"raw": raw_cols, "norm": norm_cols})[dupes].sort_values("norm"))
else:
    print("✅ Column normalization OK. Total cols:", len(norm_cols))

✅ Column normalization OK. Total cols: 65


### Step3: auto-generate the raw table DDL

In [11]:
table_name = "property_assessment_fy2024"

ddl_cols = ",\n  ".join([f'"{c}" TEXT' for c in norm_cols])

create_raw_sql = f'''
DROP TABLE IF EXISTS raw.{table_name};

CREATE TABLE raw.{table_name} (
  {ddl_cols}
);
'''.strip()

print(create_raw_sql[:1200])  # preview first chunk

DROP TABLE IF EXISTS raw.property_assessment_fy2024;

CREATE TABLE raw.property_assessment_fy2024 (
  "pid" TEXT,
  "cm_id" TEXT,
  "gis_id" TEXT,
  "st_num" TEXT,
  "st_name" TEXT,
  "unit_num" TEXT,
  "city" TEXT,
  "zip_code" TEXT,
  "bldg_seq" TEXT,
  "num_bldgs" TEXT,
  "luc" TEXT,
  "lu" TEXT,
  "lu_desc" TEXT,
  "bldg_type" TEXT,
  "own_occ" TEXT,
  "owner" TEXT,
  "mail_addressee" TEXT,
  "mail_street_address" TEXT,
  "mail_city" TEXT,
  "mail_state" TEXT,
  "mail_zip_code" TEXT,
  "res_floor" TEXT,
  "cd_floor" TEXT,
  "res_units" TEXT,
  "com_units" TEXT,
  "rc_units" TEXT,
  "land_sf" TEXT,
  "gross_area" TEXT,
  "living_area" TEXT,
  "land_value" TEXT,
  "bldg_value" TEXT,
  "sfyi_value" TEXT,
  "total_value" TEXT,
  "gross_tax" TEXT,
  "yr_built" TEXT,
  "yr_remodel" TEXT,
  "structure_class" TEXT,
  "roof_structure" TEXT,
  "roof_cover" TEXT,
  "int_wall" TEXT,
  "ext_fnished" TEXT,
  "int_cond" TEXT,
  "ext_cond" TEXT,
  "overall_cond" TEXT,
  "bed_rms" TEXT,
  "full_bth

### Step4: run DDL in Postgres

In [12]:
with engine.begin() as conn:
    conn.execute(text(create_raw_sql))

print("✅ raw table created:", f"raw.{table_name}")

✅ raw table created: raw.property_assessment_fy2024
